### 0. Imports and config

In [ ]:
%load_ext autoreload
%autoreload 2
# !pip install -e ../  # If not done yet...

In [ ]:
import os
import matplotlib.pyplot as plt
from pubrecon.data import DataFrame, ImagesData
from pubrecon.model import RCNN

In [ ]:
# General
data_path = "../data/in/"  # Where is your input data
work_path = "../data/out/"  # Where everything will be saved
seed = 1337  # Random seed
verbose = 1  # 0: no output; 1: normal informations; 2: e v e r y th i n g

# DataFrame
dataframe_pickle_path = os.path.join(work_path, "dataframe.pickle")  # Where will the DataFrame be saved
force_preparation = True  # Do you want to bypassed the saved DataFrame
subsamples = 128  # Number of samples to use for the DataFrame; -1: Use all of them

# ImagesData
imagesdata_pickle_path = os.path.join(work_path, 'imagesdata.pickle')  # Where will the ImagesData be saved
number_of_results = 2500  # How many samples will selective search use
iou_threshold = 0.85  # What is the percent of precision required
max_samples = 15  # How many class samples do you want
show_infos = True  # Show information for images output
show_labels = True  # Show labels for images output

# RCNN
model_and_weights_path = "../data/out/"  # Where will the model and weights be saved/loaded
loss = None  # Loss function; None: Use 
opt = None  # Optimization function; None: Use Adame
lr = 0.001  # Learning rate
epochs = 5  # Number of epochs
batch_size = 16
split_size = 0.15  # Test/Train proportion
checkpoint_path = os.path.join(work_path, 'checkpoint.h5')  # Where will the checkpoints be saved; None: No checkpoint (don't.)
early_stopping = True  # Should the learning stop if no more improvment is done
threshold = 0.85  # Threshold used for the recognition

### 1. DataFrame

In [ ]:
dataframe = DataFrame(data_path, pickle_path=dataframe_pickle_path)

In [ ]:
dataframe.prepare_data(force_preparation=force_preparation, subsamples=subsamples, verbose=verbose)

In [ ]:
dataframe.summary()

### 2. ImagesData

In [ ]:
imagesdata = ImagesData(dataframe, pickle_path=imagesdata_pickle_path)

In [ ]:
# That part is quite long, beware!
imagesdata.prepare_images_and_labels(number_of_results=number_of_results, iou_threshold=iou_threshold,
                                     max_samples=max_samples, seed=seed, verbose=verbose)

In [ ]:
imagesdata.summary()

### 3. Model

In [ ]:
model = RCNN(imagesdata, loss=loss, opt=opt, lr=lr, verbose=verbose)

In [ ]:
model.train(epochs=epochs, batch_size=batch_size, split_size=split_size, checkpoint_path=checkpoint_path, early_stopping=early_stopping,
            verbose=verbose)

### 4. Metrics

In [ ]:
loss = model.history()['loss']
val_loss = model.history()['val_loss']
accuracy = model.history()['acc']
val_accuracy = model.history()['val_acc']

In [ ]:
# Loss
plt.plot(loss)
plt.plot(val_loss)
plt.title("Model loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Loss", "Validation Loss"])
plt.show()

In [ ]:
print("Final loss: {}".format(loss[-1]))

In [ ]:
# Accuracy
plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title("Model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy", "Validation Accuracy"])
plt.show()

In [ ]:
print("Final accuracy: {}".format(accuracy[-1]))

### 5. Recognition example

In [ ]:
imagesdata.show_image(6, show_infos=show_infos, show_labels=show_labels)

In [ ]:
model.test_image(6, show_infos=show_infos, show_labels=show_labels, number_of_results=number_of_results,
                 threshold=threshold, verbose=verbose)